add story here

add

add

add

add


In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import folium
import requests

<h1> Importing and cleaning data </h1>

add

add

add

add

<h2> Census data </h2> 

<p> The first step in the analysis will be identifying cities where our friends could open their pizza place. We know that they'd like to relocate somewhere in California and avoid cities with less than 30k inhabitants.</p> 

<p> The best place to get a complete list of potential location is the California Census. This data is publicly available for download as an xlsx file and allows us to easily filter cities by population. </p>

<p> The main challenge will be data cleaning. An initial step has already been taken by removing all unnecessary columns directly in Excel. The next step will include removing county-level totals, empty lines, the "city" and "CPD" suffixes as well as the numerous unneeded spaces placed before and after text strings.</p>

In [2]:
census=pd.read_excel('CensusData/census.xlsx')
census.head()

,Geography,Total population
0,NaN,NaN
1,Alameda County,1510271.0
2,Alameda city,73812.0
3,Albany city,18539.0
4,Ashland CDP,21925.0


In [3]:
census.shape

(1643, 2)

In [4]:
census.dropna(inplace=True) #deleting empty lines
census.shape

(1585, 2)

In [5]:
census.columns #checking column names 

Index(['Geography', 'Total population'], dtype='object')

In [6]:
census.rename(columns={'Geography':'location','Total population':'population'}, inplace=True) #renaming columns
census.head()

,location,population
1,Alameda County,1510271.0
2,Alameda city,73812.0
3,Albany city,18539.0
4,Ashland CDP,21925.0
5,Berkeley city,112580.0


In [7]:
county_filter=~census['location'].str.contains('County') #creating a filter to eliminate rows with the count totals
census=census[county_filter]
census.head()

,location,population
2,Alameda city,73812.0
3,Albany city,18539.0
4,Ashland CDP,21925.0
5,Berkeley city,112580.0
6,Castro Valley CDP,61388.0


In [8]:
census['location']=census['location'].str.replace('CDP','')   #removing the CDP suffix
census.head()

,location,population
2,Alameda city,73812.0
3,Albany city,18539.0
4,Ashland,21925.0
5,Berkeley city,112580.0
6,Castro Valley,61388.0


In [9]:
census['location']=census['location'].str.replace('city','')  #removing the city suffix
census.head()

,location,population
2,Alameda,73812.0
3,Albany,18539.0
4,Ashland,21925.0
5,Berkeley,112580.0
6,Castro Valley,61388.0


In [10]:
census.dtypes #checking that the dataypes are correct 

location       object
population    float64
dtype: object

In [11]:
census.shape

(1527, 2)

In [12]:
#the location column is plagued with extra spaces at the beginning and ending of strings (probably placed for aestetic purposes in the original database)
#we can see an example by testing one of the location names 
census.loc[5,'location']

'    Berkeley  '

In [13]:
census['location']=census['location'].str.strip()  #removing the spaces
census.loc[5,'location'] #and testing it on one of the names

'Berkeley'

In [14]:
#we have also been asked to avoid cities with less than 30k inhabitants so we'll drop them from the dataframe
Pop_filter=(census['population']>=30000) #creating a filter 
census=census[Pop_filter] 
census.shape

(254, 2)

In [15]:
census.reset_index(inplace=True, drop=True)
census.head()

,location,population
0,Alameda,73812.0
1,Berkeley,112580.0
2,Castro Valley,61388.0
3,Dublin,46036.0
4,Fremont,214089.0


<h2> Search interest </h2> 

<p>Knowing the population is useful but, since not everyone is interested in pizza, doesn't tell us much about the potential of each location.  The ideal solution would be to conduct surveys in each location to gauge local interest but this would be prohibitively expensive for a family business and very time-consuming. </p>

<p>An alternative solution is to use freely available data from Google Trends to compare the search levels for the keyword pizza in each location. </p>

<h4> Understanding Google Trends data </h4> 

<p>Moreover, the page uses a tree-structure that makes it impossible to view more than one geographical area at a time.  For this reason, it is possible to compare the intensity of searches between metropolitan areas of the same state, and between cities of the same metropolitan area, but it is not possible to directly compare cities located in different metropolitan areas.</p>

<p>To get around this problem it was decided to first obtain the score of each metropolitan area of California and then those for every city within each metropolitan area.  The scores of each city will then be weighted using the score of their metropolitan area to get data that is directly comparable. </p>

<p>NOTE: Google also displays the variation of data over time but in this particular case the search volume was stable over time and the average value for the past year was used</p>

<h3> State-level data </h3>
<p> This data shows the search interest in each metropolitan-area (as defined by Google) of the state of California </p>

In [17]:
Google_metro=pd.read_csv('SearchData/MetropolitanData.csv')
Google_metro.reset_index(inplace=True)
Google_metro.head()

,index,Categoria: Tutte le categorie
0,DMA,pizza: (07/05/19 - 07/05/20)
1,Bakersfield CA,100
2,Eureka CA,92
3,Sacramento-Stockton-Modesto CA,89
4,Fresno-Visalia CA,88


In [18]:
Google_metro.drop([0], inplace=True) #dropping the line we don't need
Google_metro.head()

,index,Categoria: Tutte le categorie
1,Bakersfield CA,100
2,Eureka CA,92
3,Sacramento-Stockton-Modesto CA,89
4,Fresno-Visalia CA,88
5,Chico-Redding CA,87


In [19]:
Google_metro.columns=['metro_area','metro_score'] #renaming the columns 
Google_metro.head()

,metro_area,metro_score
1,Bakersfield CA,100
2,Eureka CA,92
3,Sacramento-Stockton-Modesto CA,89
4,Fresno-Visalia CA,88
5,Chico-Redding CA,87


In [20]:
Google_metro.dtypes  #checking datatypes

metro_area     object
metro_score    object
dtype: object

In [21]:
Google_metro['metro_score']=Google_metro['metro_score'].astype('int64') #casting the score to int
Google_metro.dtypes 

metro_area     object
metro_score     int64
dtype: object

<h3>Importing and cleaning data for each Metropolitan area</h3>
<p> NOTE: since all Google datasets were obtained in the same way I will apply the data cleaning process to all of them but do so in a single cell </p>

In [22]:
Bakersfield=pd.read_csv('SearchData/1_Bakersfield.csv')
Bakersfield.reset_index(inplace=True) #resetting the index - the original one creates problems
Bakersfield.drop([0], inplace=True) #deleting an unneeded line
Bakersfield.columns=['city','original_score']
Bakersfield['original_score']=Bakersfield['original_score'].astype('int64')
Bakersfield.head()

,city,original_score
1,Delano,100
2,Wasco,88
3,Bakersfield,82
4,McFarland,81
5,Arvin,76


Since the ultimate goal is to merge all the local dataframe into a single one, it will be useful to add a metropolitan area column to the dataframe to help distinguish cities with the same name. 

In [23]:
Bakersfield['metro_area']='Bakersfield'
Bakersfield.head()

,city,original_score,metro_area
1,Delano,100,Bakersfield
2,Wasco,88,Bakersfield
3,Bakersfield,82,Bakersfield
4,McFarland,81,Bakersfield
5,Arvin,76,Bakersfield


Since we will need to know the weighted score we can add a new column that multiples the original score by the weight. The weight is the score for the metropolitan area divided by 100

In [24]:
Bakersfield['weighted_score']=Bakersfield['original_score']*(Google_metro.loc[1,'metro_score']/100)
Bakersfield.head() #note, in this case nothing very exciting happens because the weight just so happens to be 1 

,city,original_score,metro_area,weighted_score
1,Delano,100,Bakersfield,100.0
2,Wasco,88,Bakersfield,88.0
3,Bakersfield,82,Bakersfield,82.0
4,McFarland,81,Bakersfield,81.0
5,Arvin,76,Bakersfield,76.0


In [25]:
Bakersfield.drop(columns=['original_score'],inplace=True) #the original score coulumn is no longer needed so we can drop it
Bakersfield.head()

,city,metro_area,weighted_score
1,Delano,Bakersfield,100.0
2,Wasco,Bakersfield,88.0
3,Bakersfield,Bakersfield,82.0
4,McFarland,Bakersfield,81.0
5,Arvin,Bakersfield,76.0


<h4>Creating a function to make importing easier</h4>
Since this process is likely to be very repetitive and the format of all the source files and file names is standardised it is easy to create a function that takes as an argument the name of the area and it's position in the list

In [26]:
def Import_Clean (Order,Place):
    filename='SearchData/{}_{}.csv'.format(Order,Place)   #generating the filename
    temp_df=pd.read_csv(filename)  #importing the csv
    temp_df.reset_index(inplace=True) #resetting the index
    temp_df.drop([0], inplace=True) #dropping first row
    temp_df.columns=['city','original_score'] #renaming columns
    temp_df['original_score']=temp_df['original_score'].astype('int64') #casting score type
    temp_df['metro_area']=Place #adding a column to keep track of the metro area
    weight=Google_metro.loc[Order,'metro_score']/100 #calculating weight
    temp_df['weighted_score']=temp_df['original_score']*weight #adding weighted score column
    temp_df.drop(columns=['original_score'],inplace=True) #dropping original score colunn 
    return temp_df

In [27]:
#testing the function
Eureka=Import_Clean(2,'Eureka')
Eureka.head(10)

,city,metro_area,weighted_score
1,Hydesville,Eureka,92.00
2,Eureka,Eureka,69.92
3,Arcata,Eureka,68.08
4,McKinleyville,Eureka,58.88


In [28]:
#importing all the other CSVs

Sacramento=Import_Clean(3,'Sacramento')
Fresno=Import_Clean(4,'Fresno')
Chico=Import_Clean(5,'Chico')
PalmSprings=Import_Clean(6,'PalmSprings')
SantaBarbara=Import_Clean(7,'SantaBarbara')
LosAngeles=Import_Clean(8,'LosAngeles')
SanDiego=Import_Clean(9,'SanDiego')
Monterey=Import_Clean(10,'Monterey')
SanFrancisco=Import_Clean(11,'SanFrancisco')
Yuma=Import_Clean(12,'Yuma')

<h3> Merging local data into a single df </h3>

In [29]:
#merging them into a single df
California=pd.concat([Bakersfield,Eureka,Sacramento,Fresno,Chico,PalmSprings,SantaBarbara,LosAngeles,SanDiego,Monterey,SanFrancisco,Yuma])
California.reset_index
California.head(10)

,city,metro_area,weighted_score
1,Delano,Bakersfield,100.00
2,Wasco,Bakersfield,88.00
3,Bakersfield,Bakersfield,82.00
4,McFarland,Bakersfield,81.00
5,Arvin,Bakersfield,76.00
6,Shafter,Bakersfield,69.00
1,Hydesville,Eureka,92.00
2,Eureka,Eureka,69.92
3,Arcata,Eureka,68.08
4,McKinleyville,Eureka,58.88


In [30]:
California.shape

(370, 3)

In [31]:
California.rename(columns={'weighted_score':'search_score'}, inplace=True)  #changing the name of the column to make clear what it contains 
California.head()

,city,metro_area,search_score
1,Delano,Bakersfield,100.0
2,Wasco,Bakersfield,88.0
3,Bakersfield,Bakersfield,82.0
4,McFarland,Bakersfield,81.0
5,Arvin,Bakersfield,76.0


In [32]:
California.dtypes

city             object
metro_area       object
search_score    float64
dtype: object

In [33]:
#now that the data has been merged we can go ahead and set the city as the index
California.reset_index(inplace=True, drop=True)
California.head()

,city,metro_area,search_score
0,Delano,Bakersfield,100.0
1,Wasco,Bakersfield,88.0
2,Bakersfield,Bakersfield,82.0
3,McFarland,Bakersfield,81.0
4,Arvin,Bakersfield,76.0


<h2> Competition </h2>

<p>Another important element is understanding how much competition exists in each location. To get an idea we'll use FourSquare's API to obtain information on the number of pizza places that can be found within a 10km radius of each location. </p>
<p>With this data, it will be possible to calculate a ratio of pizza places per inhabitant and then use this data in conjunction with search interest to conduct further analysis.</p>
<p> </p>

<h1> Analyzing data </h1>